In [21]:
using DrWatson
@quickactivate "HEMI" 
using HEMI 
using DataFrames, Chain
using CSV, StringEncodings
using Plots

## Métricas de evaluación histórica

In [22]:
## Directorios de resultados 
config_savepath = datadir("results", "absme-combination", "Esc-G")
tray_dir = datadir(config_savepath, "tray_infl")

# CountryStructure con datos hasta período de evaluación 
gtdata_eval = gtdata[Date(2020, 12)]

UniformCountryStructure{2, Float32, Float32} con 2 bases
|─> VarCPIBase{Float32, Float32}: 120 períodos × 218 gastos básicos Jan-2001-Dec-2010
|─> VarCPIBase{Float32, Float32}: 120 períodos × 279 gastos básicos Jan-2011-Dec-2020


In [23]:
df_results = collect_results(config_savepath)
df_results

┌ Info: Scanning folder c:\Users\RRCP\Documents\HEMI\HEMI\data\results\absme-combination\Esc-G for result files.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:88
┌ Info: Added 7 entries.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:118


,gt_b00_mse_bias,traindate,inflfn
,Float32?,Date?,Inflati…?
1,0.322124,2020-12-01,"CombinationFunction{3, Vector{Float32}}(EnsembleFunction{3}((InflationCoreMai{Float32, Float64, MaiFP{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiFP{Vector{Float64}}(5, [0.0, 0.27659, 0.515634, 0.705296, 0.844231, 1.0])), InflationCoreMai{Float32, Float64, MaiG{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiG{Vector{Float64}}(5, [0.0, 0.314251, 0.448574, 0.71937, 0.830707, 1.0])), InflationCoreMai{Float32, Float64, MaiF{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiF{Vector{Float64}}(10, [0.0, 0.200736, 0.227323, 0.342667, 0.420023, 0.522214, 0.613693, 0.687521, 0.746755, 0.857262, 1.0])))), Float32[0.58963, 0.379532, 0.0309246], ""MAI óptima de absme 2018"")"
2,0.0297035,2020-12-01,"InflationDynamicExclusion(1.03194, 3.42365)"
3,0.119365,2020-12-01,"InflationFixedExclusionCPI{2}(([35, 30, 190, 36, 37, 40, 31, 104, 162], [29, 116, 31, 46, 39, 40]))"
4,0.0457432,2020-12-01,"InflationTrimmedMeanEq(35.2881, 93.4009)"
5,0.168776,2020-12-01,"InflationTrimmedMeanWeighted(34.1943, 93.0)"
6,0.0616134,2020-12-01,InflationPercentileEq(0.716344)
7,0.0341601,2020-12-01,InflationPercentileWeighted(0.695585)


### Métricas de los 4 períodos

In [24]:
select(df_results, :measure, r"absme")

,measure,gt_b00_absme,gt_t0010_absme,gt_b10_absme
,String?,Float32?,Float32?,Float32?
1,MAI óptima de absme 2018,0.53724,0.726673,0.578433
2,"Inflación de exclusión dinámica (1.03, 3.42)",0.0124561,0.703433,0.073931
3,"Exclusión fija de gastos básicos IPC (9, 6)",0.0520592,0.194493,0.0983887
4,"Media Truncada Equiponderada (35.29, 93.4)",0.19482,0.914,0.276333
5,"Media Truncada Ponderada (34.19, 93.0)",0.379987,0.742717,0.176912
6,Percentil equiponderado 71.63,0.228566,1.14234,0.0191079
7,Percentil ponderado 69.56,0.0582796,1.1553,0.484528


In [25]:
select(df_results, :measure, :absme)

,measure,absme
,String?,Float32?
1,MAI óptima de absme 2018,0.0545127
2,"Inflación de exclusión dinámica (1.03, 3.42)",0.0630504
3,"Exclusión fija de gastos básicos IPC (9, 6)",0.0622681
4,"Media Truncada Equiponderada (35.29, 93.4)",0.00510523
5,"Media Truncada Ponderada (34.19, 93.0)",0.0609842
6,Percentil equiponderado 71.63,0.172761
7,Percentil ponderado 69.56,0.258382


### Métricas de la combinación óptima CORR

In [26]:
optabsme2022 = wload(datadir(config_savepath, "optabsme2022", "optabsme2022.jld2"), "optabsme2022")
components_df = components(optabsme2022)

,measure,weights
,String,Float32
1,"Media Truncada Equiponderada (35.29, 93.4)",0.407029
2,MAI óptima de absme 2018,0.0598154
3,"Media Truncada Ponderada (34.19, 93.0)",0.133195
4,"Inflación de exclusión dinámica (1.03, 3.42)",0.152838
5,Percentil equiponderado 71.63,0.164859
6,Percentil ponderado 69.56,0.0822618
7,"Exclusión fija de gastos básicos IPC (9, 6)",0.0


In [27]:
# DataFrame de combinación 
combine_df = @chain df_results begin 
    select(:measure, :absme, :me, :inflfn, 
        :path => ByRow(p -> joinpath(tray_dir, basename(p))) => :tray_path)
    sort(:absme)
end
combine_df = leftjoin(combine_df, components_df, on = :measure)
combine_df

## Obtener las trayectorias de los archivos guardados en el directorio tray_infl 
# Genera un arreglo de 3 dimensiones de trayectorias (T, n, K)
tray_infl = mapreduce(hcat, combine_df.tray_path) do path
    load(path, "tray_infl")
end

functions = combine_df.inflfn
components_mask = [!(fn isa InflationFixedExclusionCPI) for fn in functions];

In [28]:
# Obtener el parámetro
resamplefn = df_results[1, :resamplefn]
trendfn = df_results[1, :trendfn]
paramfn = df_results[1, :paramfn]
param = InflationParameter(paramfn, resamplefn, trendfn)
tray_infl_pob = param(gtdata_eval);

In [29]:
# Pesos de la combinación óptima
absme_weights = combine_df.weights

EVAL_PERIODS = [GT_EVAL_B00, GT_EVAL_T0010, GT_EVAL_B10, CompletePeriod()]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* absme_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end;


In [30]:
# Obtener un DataFrame de métricas de la combinación lineal ABMSE
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optabsme2022)
select(comb_metrics_df, :measure, r"absme")

,measure,absme,gt_b00_absme,gt_b10_absme,gt_t0010_absme
,String,Float32,Float32,Float32,Float32
1,Subyacente óptima ABSME 2022,0.0565886,0.0275331,4.93534e-6,0.905293


In [31]:
historic_df = [
    select(df_results, :measure, r"absme");
    select(comb_metrics_df, :measure, r"absme")
]

,measure,gt_b00_absme,gt_t0010_absme,gt_b10_absme
,String?,Float32?,Float32?,Float32?
1,MAI óptima de absme 2018,0.53724,0.726673,0.578433
2,"Inflación de exclusión dinámica (1.03, 3.42)",0.0124561,0.703433,0.073931
3,"Exclusión fija de gastos básicos IPC (9, 6)",0.0520592,0.194493,0.0983887
4,"Media Truncada Equiponderada (35.29, 93.4)",0.19482,0.914,0.276333
5,"Media Truncada Ponderada (34.19, 93.0)",0.379987,0.742717,0.176912
6,Percentil equiponderado 71.63,0.228566,1.14234,0.0191079
7,Percentil ponderado 69.56,0.0582796,1.1553,0.484528
8,Subyacente óptima ABSME 2022,0.0275331,0.905293,4.93534e-6


## Métricas en período de optimización (Dic-2001 - Dic-2018)

In [32]:
## Directorios de resultados 
config_savepath = datadir("results", "absme-combination", "Esc-G18")
tray_dir = datadir(config_savepath, "tray_infl")

# CountryStructure con datos hasta período de evaluación 
gtdata_eval = gtdata[Date(2018, 12)]

# DataFrame con resultados de evaluación hasta dic-18
df_results = collect_results(config_savepath)
df_results

┌ Info: Scanning folder c:\Users\RRCP\Documents\HEMI\HEMI\data\results\absme-combination\Esc-G18 for result files.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:88


┌ Info: Added 7 entries.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:118


,gt_b00_mse_bias,traindate,inflfn
,Float32?,Date?,Inflati…?
1,0.230244,2018-12-01,"CombinationFunction{3, Vector{Float32}}(EnsembleFunction{3}((InflationCoreMai{Float32, Float64, MaiFP{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiFP{Vector{Float64}}(5, [0.0, 0.27659, 0.515634, 0.705296, 0.844231, 1.0])), InflationCoreMai{Float32, Float64, MaiG{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiG{Vector{Float64}}(5, [0.0, 0.314251, 0.448574, 0.71937, 0.830707, 1.0])), InflationCoreMai{Float32, Float64, MaiF{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiF{Vector{Float64}}(10, [0.0, 0.200736, 0.227323, 0.342667, 0.420023, 0.522214, 0.613693, 0.687521, 0.746755, 0.857262, 1.0])))), Float32[0.58963, 0.379532, 0.0309246], ""MAI óptima de absme 2018"")"
2,0.0297035,2018-12-01,"InflationDynamicExclusion(1.03194, 3.42365)"
3,0.119365,2018-12-01,"InflationFixedExclusionCPI{2}(([35, 30, 190, 36, 37, 40, 31, 104, 162], [29, 116, 31, 46, 39, 40]))"
4,0.0457432,2018-12-01,"InflationTrimmedMeanEq(35.2881, 93.4009)"
5,0.168776,2018-12-01,"InflationTrimmedMeanWeighted(34.1943, 93.0)"
6,0.0616134,2018-12-01,InflationPercentileEq(0.716344)
7,0.0341601,2018-12-01,InflationPercentileWeighted(0.695585)


In [33]:
# Se utiliza solamente :corr porque se desea únicament el período completo de
# estos datos (hasta dic-18) para obtener las métricas de optimización
select(df_results, :measure, :absme)

,measure,absme
,String?,Float32?
1,MAI óptima de absme 2018,3.56707e-7
2,"Inflación de exclusión dinámica (1.03, 3.42)",0.000845995
3,"Exclusión fija de gastos básicos IPC (9, 6)",0.00255378
4,"Media Truncada Equiponderada (35.29, 93.4)",0.000175531
5,"Media Truncada Ponderada (34.19, 93.0)",0.127516
6,Percentil equiponderado 71.63,0.150155
7,Percentil ponderado 69.56,0.184207


### Métricas de la combinación óptima CORR en período de optimización

In [34]:
# DataFrame de combinación 
combine_df = @chain df_results begin 
    select(:measure, :absme, :me, :inflfn, 
        :path => ByRow(p -> joinpath(tray_dir, basename(p))) => :tray_path)
    sort(:absme)
end
combine_df = leftjoin(combine_df, components_df, on = :measure)
combine_df

## Obtener las trayectorias de los archivos guardados en el directorio tray_infl 
# Genera un arreglo de 3 dimensiones de trayectorias (T, n, K)
tray_infl = mapreduce(hcat, combine_df.tray_path) do path
    load(path, "tray_infl")
end

functions = combine_df.inflfn
components_mask = [!(fn isa InflationFixedExclusionCPI) for fn in functions];

In [35]:
# Obtener el parámetro
resamplefn = df_results[1, :resamplefn]
trendfn = df_results[1, :trendfn]
paramfn = df_results[1, :paramfn]
param = InflationParameter(paramfn, resamplefn, trendfn)
tray_infl_pob = param(gtdata_eval);

In [38]:
# Pesos de la combinación óptima
absme_weights = combine_df.weights

EVAL_PERIODS = [EvalPeriod(Date(2001, 12), Date(2018, 12), "opt18")]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* absme_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end

# Obtener un DataFrame de métricas de la combinación lineal ABSME
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optabsme2022)
select(comb_metrics_df, :measure, r"absme")

,measure,opt18_absme
,String,Float32
1,Subyacente óptima ABSME 2022,0.0231238


In [39]:
# Combinar todas las métricas en período de optimización 
optim_period_df = [
    select(df_results, :measure, :absme => :opt18_absme); 
    select(comb_metrics_df, :measure, r"absme")
]

,measure,opt18_absme
,String?,Float32?
1,MAI óptima de absme 2018,3.56707e-7
2,"Inflación de exclusión dinámica (1.03, 3.42)",0.000845995
3,"Exclusión fija de gastos básicos IPC (9, 6)",0.00255378
4,"Media Truncada Equiponderada (35.29, 93.4)",0.000175531
5,"Media Truncada Ponderada (34.19, 93.0)",0.127516
6,Percentil equiponderado 71.63,0.150155
7,Percentil ponderado 69.56,0.184207
8,Subyacente óptima ABSME 2022,0.0231238


## Combinar las métricas en un DataFrame de salida

In [43]:
final_df = leftjoin(historic_df, optim_period_df, on=:measure)
select(final_df, :measure, :gt_b10_absme, :absme, :opt18_absme)

,measure,gt_b10_absme,absme,opt18_absme
,String?,Float32?,Float32?,Float32?
1,MAI óptima de absme 2018,0.578433,0.0545127,3.56707e-7
2,"Inflación de exclusión dinámica (1.03, 3.42)",0.073931,0.0630504,0.000845995
3,"Exclusión fija de gastos básicos IPC (9, 6)",0.0983887,0.0622681,0.00255378
4,"Media Truncada Equiponderada (35.29, 93.4)",0.276333,0.00510523,0.000175531
5,"Media Truncada Ponderada (34.19, 93.0)",0.176912,0.0609842,0.127516
6,Percentil equiponderado 71.63,0.0191079,0.172761,0.150155
7,Percentil ponderado 69.56,0.484528,0.258382,0.184207
8,Subyacente óptima ABSME 2022,4.93534e-6,0.0565886,0.0231238


In [44]:
# Guardar los resultados en un archivo CSV
function save_csv(file::AbstractString, df::DataFrame)
    encoding = enc"ISO-8859-1"
    @info "Saving file with $encoding" file
    open(file, encoding, "w") do io 
        CSV.write(io, df)
    end
end

metrics_savepath = mkpath(datadir("updates", "metrics"))
save_csv(
    joinpath(metrics_savepath, "optabsme2022_metrics.csv"),
    final_df
)

┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\metrics\optabsme2022_metrics.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\absme-combination-metrics.ipynb:4


StringEncoder{UTF-8, ISO-8859-1}(IOStream(<file c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\metrics\optabsme2022_metrics.csv>))